# Anormal Durum (Aşırı CPU Kullanımı) Tespit Modülü

In [1]:
# --- Genel Kütüphaneler ---
import pandas as pd
import numpy as np
import os
import plotly.express as px

# --- Tespit Modeli İçin Kütüphaneler ---
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Adım 1: Veri Yükleme ve Etiketleme

In [ ]:
print("Anormallik tespiti için ham veri yükleniyor...")
DATA_PATH = 'data'
# Analiz için ilk 5 dosya 
FILES_TO_READ = [f"vm_cpu_readings-file-{i}-of-195.csv.gz" for i in range(1, 6)]
COLUMN_NAMES = ['timestamp', 'vmId', 'min_cpu', 'max_cpu', 'avg_cpu']

df_list = []

for file_name in FILES_TO_READ:
    full_path = os.path.join(DATA_PATH, file_name)
    if os.path.exists(full_path):
        temp_df = pd.read_csv(full_path, header=None, names=COLUMN_NAMES, compression='gzip')
        df_list.append(temp_df)
        print(f"- '{file_name}' okundu.")

if df_list:
    df_raw = pd.concat(df_list, ignore_index=True)
    print("\n--- Tüm ham veriler başarıyla birleştirildi! ---")
    df_raw.info()
else:
    print("\nOkunacak veri dosyası bulunamadı. Lütfen 'data' klasörünü kontrol edin.")

Anormallik tespiti için ham veri yükleniyor...
- 'vm_cpu_readings-file-1-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-2-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-3-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-4-of-195.csv.gz' okundu.
- 'vm_cpu_readings-file-5-of-195.csv.gz' okundu.

--- Tüm ham veriler başarıyla birleştirildi! ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000000 entries, 0 to 49999999
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   vmId       object 
 2   min_cpu    float64
 3   max_cpu    float64
 4   avg_cpu    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.9+ GB


## Adım 2: Veri Etiketleme ve Hazırlama

In [ ]:
# Etiketleme için bir eşik değeri
anomaly_threshold = 90  # %90 CPU kullanımı

# Yeni bir 'etiket' sütunu oluşturalım (1 = Anormal, 0 = Normal)
df_raw['etiket'] = (df_raw['avg_cpu'] >= anomaly_threshold).astype(int)

print("Etiket Dağılımı:")
print(df_raw['etiket'].value_counts())

# Özellik (Feature) ve Hedef (Target) Belirleme
X = df_raw[['min_cpu', 'max_cpu', 'avg_cpu']]
y = df_raw['etiket']

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("\nEğitim seti boyutu:", X_train.shape)
print("Test seti boyutu:", X_test.shape)

Etiket Dağılımı:
etiket
0    49606582
1      393418
Name: count, dtype: int64

Eğitim seti boyutu: (35000000, 3)
Test seti boyutu: (15000000, 3)


## Adım 3: Karar Ağacı Modelini Kurma, Eğitme ve Değerlendirme

In [4]:
# Karar Ağacı Sınıflandırıcısı modelini oluşturalım ve eğitelim
# max_depth parametresi, ağacın ne kadar derinleşeceğini kontrol ederek ezberlemeyi (overfitting) önler.
clf = DecisionTreeClassifier(max_depth=5, random_state=42)

print("Karar Ağacı modeli eğitiliyor...")
clf.fit(X_train, y_train)
print("Eğitim tamamlandı.")

# Modelin test verisi üzerinde tahmin yapmasını sağlayalım
y_pred = clf.predict(X_test)

# Modelin performansını metriklerle değerlendirelim
print("\n--- Model Performansı ---")
print(f"Doğruluk (Accuracy): {accuracy_score(y_test, y_pred):.3f}")

# Sınıflandırma Raporu (Precision, Recall, F1-Score)
# Bu metrikler, özellikle 'Anormal' sınıfı gibi az sayıda olan sınıflar için daha anlamlıdır.
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred, target_names=['Normal (0)', 'Anormal (1)']))

# Karmaşıklık Matrisi (Confusion Matrix)
# Modelin neleri doğru, neleri yanlış sınıflandırdığını detaylıca gösterir.
print("\nKarmaşıklık Matrisi:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

Karar Ağacı modeli eğitiliyor...
Eğitim tamamlandı.

--- Model Performansı ---
Doğruluk (Accuracy): 1.000

Sınıflandırma Raporu:
              precision    recall  f1-score   support

  Normal (0)       1.00      1.00      1.00  29784524
 Anormal (1)       1.00      1.00      1.00    215476

    accuracy                           1.00  30000000
   macro avg       1.00      1.00      1.00  30000000
weighted avg       1.00      1.00      1.00  30000000


Karmaşıklık Matrisi:
[[29784524        0]
 [       0   215476]]


## Adım 4: Rastgele Orman Modelini Kurma, Eğitme ve Değerlendirme

In [4]:

# Random Forest Sınıflandırıcısı modelini oluşturalım
# n_estimators: Ormandaki ağaç sayısı
# max_depth: Her bir ağacın maksimum derinliği
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

print("Random Forest modeli eğitiliyor...")
# Modeli eğitim verisiyle eğitelim
rf_clf.fit(X_train, y_train)
print("Eğitim tamamlandı.")

# Modelin test verisi üzerinde tahmin yapmasını sağlayalım
y_pred_rf = rf_clf.predict(X_test)

# Modelin performansını metriklerle değerlendirelim
print("\n--- Random Forest Model Performansı ---")
print(f"Doğruluk (Accuracy): {accuracy_score(y_test, y_pred_rf):.3f}")
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred_rf, target_names=['Normal (0)', 'Anormal (1)']))
print("\nKarmaşıklık Matrisi:")
print(confusion_matrix(y_test, y_pred_rf))

Random Forest modeli eğitiliyor...
Eğitim tamamlandı.

--- Random Forest Model Performansı ---
Doğruluk (Accuracy): 1.000

Sınıflandırma Raporu:
              precision    recall  f1-score   support

  Normal (0)       1.00      1.00      1.00  14881975
 Anormal (1)       1.00      1.00      1.00    118025

    accuracy                           1.00  15000000
   macro avg       1.00      1.00      1.00  15000000
weighted avg       1.00      1.00      1.00  15000000


Karmaşıklık Matrisi:
[[14881975        0]
 [       1   118024]]
